# Query Model Repository for best node embeddings model

In [1]:
from hops import model
from hops.model import Metric
MODEL_NAME="NodeEmbeddings"
EVALUATION_METRIC="accuracy"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
8,application_1607799579196_0009,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MAX)

In [3]:
best_model['experimentId']

'application_1607799579196_0006_1'

# Define model and load wights 

In [4]:
import tensorflow as tf
from tensorflow import keras  

import pandas as pd
from stellargraph import StellarDiGraph
from stellargraph.mapper import Node2VecLinkGenerator, Node2VecNodeGenerator
from stellargraph.data import UnsupervisedSampler, BiasedRandomWalk
from stellargraph.layer import Node2Vec
import pydoop.hdfs as pydoop
import hsfs
from hops import hdfs

In [5]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [6]:
node_td = fs.get_training_dataset("node_td", 1)
edge_td = fs.get_training_dataset("edges_td", 1)

In [7]:
# Get fg as pandas
node_pdf = node_td.read().toPandas()
edge_pdf = edge_td.read().drop("tran_timestamp").toPandas()

In [8]:
node_data = pd.DataFrame(node_pdf[['tx_behavior_id','prior_sar','initial_deposit']], index=node_pdf['id'])

print('Defining StellarDiGraph')
G =StellarDiGraph(node_data,
                      edges=edge_pdf, 
                      edge_type_column="tx_type")


Defining StellarDiGraph

In [27]:
from hops import experiment
from hops import hdfs
import json
best_hyperparams_path = "Resources/embeddings_best_hp.json"
best_hyperparams = json.loads(hdfs.load(best_hyperparams_path))
args_dict = {}
for key in best_hyperparams.keys():
    args_dict[key] = [best_hyperparams[key]]

In [30]:
walk_number = 2
walk_length = 2
batch_size = 1
emb_size = args_dict['emb_size'][0]
# Extracting node embeddings
walker = BiasedRandomWalk(
        G,
        n=walk_number,
        length=walk_length,
        p=0.5,  # defines probability, 1/p, of returning to source node
        q=2.0,  # defines probability, 1/q, for moving to a node away from the source node
    )
unsupervised_samples = UnsupervisedSampler(G, nodes=list(G.nodes()), walker=walker)
generator = Node2VecLinkGenerator(G, batch_size)

node2vec = Node2Vec(emb_size, generator=generator)
x_inp, x_out = node2vec.in_out_tensors()

x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [31]:
latest = tf.train.latest_checkpoint("hdfs:///Projects/{}/Experiments/".format(hdfs.project_name()) + best_model['experimentId'])
embedding_model.load_weights(latest)

In [32]:
nodes = list(G.nodes())
node_gen = Node2VecNodeGenerator(G, batch_size).flow(nodes)

In [53]:
import numpy as np
import pandas as pd 

pdf = pd.DataFrame(embedding_model.predict(node_gen), index=G.nodes())
emb_feature_names = ["em_" + str(c)  for c in pdf.columns]
pdf.columns = emb_feature_names
pdf['id'] = pdf.index
node_embeddings_df = spark.createDataFrame(pdf)

In [54]:
node_embeddings_df.show(2)

+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+------------------+------------------+-------------------+--------------------+----+
|               em_0|               em_1|               em_2|               em_3|               em_4|               em_5|              em_6|               em_7|               em_8|                em_9|              em_10|               em_11|               em_12|               em_13|              em_14|             em_15|             em_16|              em_17|               em_18|  id|
+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+-

### Create a connection to hsfs

In [55]:
import hsfs
from hops import hdfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

### Retrieve nodes training dataset from hsfs to get label infamation, whether node was part of the previously known money laundering scheme or not 

In [56]:
from pyspark.sql.functions import array, coalesce, concat,  col

node_td = fs.get_training_dataset("node_td", 1)
node_labels_df = node_td.read().select("id","is_sar")
#node_labels_df = node_labels_df.select(*(col(c).cast("float").alias(c) for c in node_labels_df.columns))

In [57]:
node_labels_df.show(5)

+----+------+
|  id|is_sar|
+----+------+
|7605|     1|
|6075|     1|
|5814|     1|
|9360|     1|
|4626|     1|
+----+------+
only showing top 5 rows

In [58]:
node_embeddings_df = node_embeddings_df.join(node_labels_df,['id'])

In [59]:
node_embeddings_df.count()

9998

In [60]:
emb_td = node_embeddings_df.drop("id").withColumn("embedding", array(emb_feature_names)).select("is_sar","embedding").withColumnRenamed("is_sar","target")

In [61]:
emb_td.show()

+------+--------------------+
|target|           embedding|
+------+--------------------+
|     1|[-0.7713622450828...|
|     1|[0.46240988373756...|
|     1|[-0.9044333696365...|
|     1|[0.80935311317443...|
|     1|[-0.0774855017662...|
|     1|[0.05340209975838...|
|     1|[0.12839069962501...|
|     1|[-0.4400345087051...|
|     1|[-0.9107171297073...|
|     0|[-0.6036044955253...|
|     0|[0.33578562736511...|
|     0|[-0.8345335721969...|
|     0|[-0.6865931153297...|
|     0|[-0.7855605483055...|
|     0|[-0.4209088087081...|
|     0|[0.21963755786418...|
|     0|[-0.2506932318210...|
|     0|[0.23866248130798...|
|     0|[-1.1010141372680...|
|     0|[-0.2126887291669...|
+------+--------------------+
only showing top 20 rows

## Prepare training datasets for anomaly detection 
###### In the next notebook we are going to train [gan for anomaly detection](https://arxiv.org/pdf/1905.11034.pdf). Durring training step  we will provide only features of accounts that have never been reported for money laundering behaviour.  But we will disclose previously reported accounts to the model only in evaluation step.   

In [62]:
non_sar_df = emb_td.where(col("target")==0)

In [63]:
sar_df = emb_td.where(col("target")==1)

In [64]:
# Now that the data has been prepared, let's split the dataset into a training and test dataframe
[non_sar_train_df, non_sar_eval_df] = non_sar_df.randomSplit([0.8, 0.02],seed = 12345)

In [65]:
non_sar_td = fs.create_training_dataset(name="gan_non_sar_training_df",
                                       version=1,
                                       data_format="tfrecord",
                                       label=["target"], 
                                       statistics_config=False, 
                                       description="non sar dataset for gan training")
non_sar_td.save(non_sar_train_df)

In [66]:
eval_df = non_sar_eval_df.union(sar_df)

In [67]:
gan_eval_ds = fs.create_training_dataset(name="gan_eval_df",
                                       version=1,
                                       data_format="tfrecord",
                                       label=["target"], 
                                       statistics_config=False, 
                                       description="evaluation dataset for gan training")
gan_eval_ds.save(eval_df)